# Introduction

Le but de ce document est d’expliquer une démonstration du théorème de Cantor-Berntein : "S'il existe une injection d'un ensemble $E$ vers un ensemble $F$ et une injection de $F$ vers $E$, alors il existe une bijection de $E$ sur $F$."

Puis de créer un programme qui génère une bijection à partir des deux injections en s’inspirant de cette démonstration, et enfin de s’en servir sur un exemple. (solution non générale)

# Explication avec les mains

Nommons $f$ l’injection de $E$ sur $F$ et $g$ l’injection de $F$ sur $E$. On va essayer de construire une fonction $h$ bijective de $E$ dans $F$. 

Pour commencer on va utiliser $g^{-1}$ qui est bien surjective (car $g$ est défini sur tout $F$) et injective car c’est une fonction inverse.  
Le problème est que cette fonction n’est pas défini sur tout $E$ (car $g$ n’était pas surjective). On utilisera alors $f$ pour l’ensemble manquant pour couvrir tout $E$, qu’on nommera $C_0=E \backslash g(F)$. 

On a donc $
h(x) =
\begin{cases}
      f(x) & si,x\in C_0 \\
      u^{-1}(x) & sinon \\
\end{cases}
$

$h$ est bien définit sur tout $E$ et surjective par construction, le problème qu’on a maintenant est que $f$ va rentrer en collision avec $g^{-1}$ sur $F$ car $g^{-1}$ occupait déjà tout $F$.  
Pour régler ce problème on va prendre toutes les valeurs $y$ de l’ensemble $F$ où il y a une collision (qui est en réalité tout $f(C_0)$, puis regarder l’antécédent de $y$ par la fonction $g^{-1}$ (qui est donc $g(y)$), et ne plus utiliser $g^{-1}$ pour ce point là. (car il est déjà couvert par $f$).

On obtient ainsi un nouvel ensemble $C_1=g(f(C_0))$

![schéma](images/schema1.png)

Notre fonction $h$ devient donc :

$h(x) =
\begin{cases}
      f(x) & si,x\in C_0 \cup C_1 \\
      u^{-1}(x) & sinon \\
\end{cases}
$

On a réglé le problème de collision pour $C_0$, malheureusement on se retrouve avec exactement le même problème pour $C_1$. Ouais je sais nous voilà bien avancé, mais pas de panique (si si vous paniquiez je l’ai senti). 

On ne se laisse par abattre et on répète cette opération une infinité de fois ($Cn+1=g(f(Cn))$).  
Et vue que quand même l’infini c’est beaucoup, ça va bien finir par marcher ! (d’accord j’arrête, partez pas !)

Plus sérieusement, si maintenant je prend un point $x$ quelconque sur $F$ il y a deux possibilités :
- Soit il n’y a jamais eu de collisions, et il n’y a donc pas de problèmes.
- Soit il y a eu une collision à une des étapes, or cette collision a donc été corrigée à l’étape suivante, et il n’y a donc plus de problèmes à partir de là. (on ne peut pas se retrouver deux fois avec la même collision)

Donc $h$ est bijective !

$h(x) =
\begin{cases}
      f(x) & si,x\in \bigcup _{i=0}^\infty C_n \\
      u^{-1}(x) & sinon \\
\end{cases}
$

D’une certaine manière on n’a fait que repousser le problème toujours plus loin à chaque fois, mais repousser le problème plus loin une infinité de fois, c’est le faire disparaitre.

Maintenant place à la démonstration.

# Démonstration

$f$ l’injection de $E$ sur $F$ et $g$ l’injection de $F$ sur $E$. On va essayer de construire une fonction $h$ bijective de $E$ dans $F$.

$C_0=E \backslash g(F)$  
$C_n+1 = g(f(C_n))$  
$C = \bigcup _{i=0}^\infty C_n$

$h(x) =
\begin{cases}
      f(x) & si x \in C \\
      u^{-1}(x) & sinon \\
\end{cases}
$

si $(x, y) \in C^2, h(x) = h(y) \Rightarrow f(x)=f(y) \Rightarrow x=y$  
si $x \notin C$ et $y \notin, h(x) = h(y) \Rightarrow g^{-1}(x)=g^{-1}(y) \Rightarrow x=y$  
si $x \notin C$ et $y \in C, h(x)=h(y) \Rightarrow g^{-1}(x)=f(y) \Rightarrow x=g(f(y)) \Rightarrow x \in C \Rightarrow$ contradiction et mensonge

Donc $h$ injective.

(Démonstration de la surjection à finir)

# Le programme

In [23]:
# Just some julia package
using Pkg
Pkg.add("Primes")
Pkg.add("SaferIntegers");

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [31]:
"Check if x is in the domain of f. (ie check than f(x) is well defined)"
function in_domain(f, x)
    try
        y = f(x)
    catch error
        if isa(error, DomainError)
            return false
        else
            rethrow()
        end
    end
    
    return true
end

function create_h((f, finv), (g, ginv))
    # C0 = E - g(f)
    in_C0(x) = !in_domain(ginv, x)
    
    function in_Cn(x)
        if in_C0(x)
            return true
        end
        
        y = ginv(x)
        if in_domain(finv, y)
            old_x = finv(y)
            return in_Cn(old_x)
        else
            return false
        end
    end
    
    h(x) = in_Cn(x) ? f(x) : ginv(x) 
    hinv(x) = in_domain(finv, x) && in_Cn(finv(x)) ? finv(x) : g(x)
    return (h, hinv)
end



create_h (generic function with 1 method)

In [38]:
using Primes
using SaferIntegers

S = NTuple{N,Int} where N
"An injection from S to ℕ (where S is the set of all finite sequences)"
function f(x::S)::Int
    if isempty(x)
        return 0
    end
    res = SafeInt(1)
    for (i, xi) in enumerate(x)
        ti = SafeInt(prime(i))^(xi+1)
        res *= ti
    end
    return res
end

"An basic injection from ℕ to S"
g(x::Int)=(x,)

ginv(x::S)=length(x) == 1 ? x[1] : throw(DomainError("ginv only defined for singleton"));

function finv(x::Int)
    if x == 0
        return ()
    end
    if x == 1
        throw(DomainError("finv(1) undefined (∄x, f(x)==1)"))
    end
    factors = factor(x)
    res = []
    for (i, f) in enumerate(factors)
        if f[1] != prime(i)
            throw(DomainError("finv($x) undefined (∄x, f(x)==$x)"))
        end
        push!(res, f[2]-1)
    end
    return Tuple(res)
end


(h, hinv) = create_h((f, finv), (g, ginv))

for x in 0:20
    y = h((x,))
    @eval @show h(($x,));
    @eval @show hinv($y);
end

println("-"^50)

for y in 0:20
    @eval @show hinv($y)
end

h((0,)) = 2
hinv(2) = (0,)
h((1,)) = 1
hinv(1) = (1,)
h((2,)) = 8
hinv(8) = (2,)
h((3,)) = 3
hinv(3) = (3,)
h((4,)) = 4
hinv(4) = (4,)
h((5,)) = 5
hinv(5) = (5,)
h((6,)) = 128
hinv(128) = (6,)
h((7,)) = 7
hinv(7) = (7,)
h((8,)) = 512
hinv(512) = (8,)
h((9,)) = 9
hinv(9) = (9,)
h((10,)) = 10
hinv(10) = (10,)
h((11,)) = 11
hinv(11) = (11,)
h((12,)) = 8192
hinv(8192) = (12,)
h((13,)) = 13
hinv(13) = (13,)
h((14,)) = 14
hinv(14) = (14,)
h((15,)) = 15
hinv(15) = (15,)
h((16,)) = 16
hinv(16) = (16,)
h((17,)) = 17
hinv(17) = (17,)
h((18,)) = 524288
hinv(524288) = (18,)
h((19,)) = 19
hinv(19) = (19,)
h((20,)) = 20
hinv(20) = (20,)
--------------------------------------------------
hinv(0) = ()
hinv(1) = (1,)
hinv(2) = (0,)
hinv(3) = (3,)
hinv(4) = (4,)
hinv(5) = (5,)
hinv(6) = (0, 0)
hinv(7) = (7,)
hinv(8) = (2,)
hinv(9) = (9,)
hinv(10) = (10,)
hinv(11) = (11,)
hinv(12) = (1, 0)
hinv(13) = (13,)
hinv(14) = (14,)
hinv(15) = (15,)
hinv(16) = (16,)
hinv(17) = (17,)
hinv(18) = (0, 1)
hinv(19) = (1